In [ ]:
!sudo apt-get install mpich

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124  # PyTorch libraries for deep learning, including vision and audio processing, optimized for CUDA 12.4
!pip install 'transformers==4.51.3'  # Library for state-of-the-art natural language processing models, including BERT, GPT, and others
!pip install 'ninja==1.11.1.4'  # Build system designed for speed, useful for compiling extensions quickly
!pip install 'wheel==0.37.1'  # Python package format for faster installation and distribution
!pip install 'flash-attn==2.7.4.post1' --no-build-isolation  # Efficient attention mechanism implementation for transformer models, optimized for speed and memory usage
!pip install 'accelerate==1.6.0'  # Library to simplify distributed training and inference of machine learning models
!pip install 'huggingface_hub[cli]==0.30.2'  # Client library and CLI for interacting with Hugging Face Hub, a platform for sharing ML models and datasets
!pip install 'bitsandbytes==0.45.5'  # Library for efficient quantization and memory optimization in deep learning models
!pip install 'peft==0.15.2'  # Parameter-efficient fine-tuning library for transformer models
!pip install 'python-dotenv==1.1.0'  # Library for managing environment variables from .env files
!pip install 'wandb==0.19.10'  # For machine learning experiment tracking and visualization
!pip install 'GPUtil==1.4.0'  # For GPU monitoring and manual logging of GPU utilization
!pip install 'psutil==7.0.0'  # For monitoring system resources and manual logging of CPU and memory usage
!pip install 'datasets==3.5.0'  # Library for easily accessing and managing datasets for machine learning
!pip install 'numpy==2.1.2'  # Fundamental library for numerical computations in Python, supports arrays and mathematical operations
!pip install 'ipywidgets==8.1.6'  # Library for creating interactive widgets in Jupyter Notebook
!pip install 'pandas==2.2.3'  # Library for data analysis and manipulation, provides DataFrame for tabular data
!pip install 'matplotlib==3.10.1'  # Library for data visualization, can draw various graphs such as line and scatter plots
!pip install 'seaborn==0.13.2'  # Data visualization library based on Matplotlib, easily creates statistical graphs
!pip install 'scipy==1.15.2'  # Library for scientific and technical computing, specialized in advanced numerical calculations and optimization
!pip install 'PyPDF2==3.0.1'  # Library for reading and writing PDF files, can extract and merge pages
!pip install 'openai==1.76.0'  # Library for using OpenAI APIs, can utilize GPT models and other AI services
!pip install 'langchain==0.3.24'  # Library for developing applications utilizing language models, supports chaining and prompt management
!pip install 'langgraph==0.3.34'  # Library for graph structure analysis and workflows involving language data and models
!pip install 'psycopg2==2.9.10'  # PostgreSQL database adapter for Python, enables database connectivity and operations
!pip install 'deepspeed==0.16.7' --use-pep517  # Deep learning optimization library for distributed training, supports large-scale models and efficient resource utilizationa
!pip install 'pyarrow==19.0.1'
!pip install 'setuptools==68.1.2' # if python-venv version==3.12~ . install 'setuptools==68.1.2'
!pip install mpi4py

Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached mpi4py-4.0.3.tar.gz (466 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp312-cp312-linux_x86_64.whl size=4931750 sha256=1ed8962038d8823a7aaf392ac1e770e77ab223876852f440875bed0f3bd14f36
  Stored in directory: /home/gotech/.cache/pip/wheels/10/44/e1/03c799a415ad873a19c5b830e6ae3c2ac6064dff0b9b2aeb7d
Successfully built mpi4py


In [2]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from transformers.integrations import is_deepspeed_available
import json
from datasets import load_dataset, Dataset


[2025-06-19 17:20:16,400] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: -laio が見つかりません: そのようなファイルやディレクトリはありません
collect2: error: ld returned 1 exit status
/usr/bin/ld: -lcufile が見つかりません: そのようなファイルやディレクトリはありません
collect2: error: ld returned 1 exit status


In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

In [4]:
# from google.colab import drive

# drive.mount('/content/drive')

In [5]:
import pandas as pd
df = pd.read_json("deepspeed_config_zero2.json", encoding="UTF-8")
df.head()

,zero_optimization,train_batch_size,eval_batch_size
stage,2,1,1


In [6]:
df = pd.read_json("alpaca_data.json", encoding="UTF-8")
df.head()

,instruction,input,output
0,Give three tips for staying healthy.,,1.Eat a balanced diet and make sure to include...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain..."
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...
4,Describe a time when you had to make a difficu...,,I had to make a difficult decision when I was ...


In [7]:
class TrainUtil:

  def __init__(self):
      self.alpaca_dataset = "alpaca_data.json"
      self.dataset_max_length = 2048

  def read_alpaca_json_dataset(self) -> list:
      with open(self.alpaca_dataset, 'r', encoding='utf-8') as f:
          dataset = json.load(f)
          for i in range(len(dataset[0:100])):
              print(f"dataset[{i}]: {dataset[i]}")
      return dataset

  def load_alpaca_dataset(self, tokenizer):
      dataset = self.read_alpaca_json_dataset()
      dataset = Dataset.from_list(dataset)
      totaL_size = total_size = int(len(dataset) * 0.2)
      dataset = dataset.select(range(total_size))
      train_size = int(total_size * 0.8)
      validation_size = int(total_size * 0.1)
      train_dataset = dataset.select(range(0, train_size))
      validation_dataset = dataset.select(range(train_size, train_size + validation_size))
      test_dataset = dataset.select(range(train_size + validation_size, total_size))
      return train_dataset, validation_dataset, test_dataset

  def load_alpaca_train_dataset(self, tokenizer: AutoTokenizer, train_dataset):
      dataset = train_dataset
      dataset = dataset.map(self.format_alpaca_dataset)
      tokenized_dataset = dataset.map(lambda example: self.alpaca_tokenize_function(example, tokenizer), batched=True)
      return tokenized_dataset

  def load_alpaca_validation_dataset(self, tokenizer: AutoTokenizer, validation_dataset):
      dataset = validation_dataset
      dataset = dataset.map(self.format_alpaca_dataset)
      tokenized_dataset = dataset.map(lambda example: self.alpaca_tokenize_function(example, tokenizer), batched=True)
      return tokenized_dataset

  def load_alpaca_test_dataset(self, tokenizer: AutoTokenizer, test_dataset):
      dataset = test_dataset
      dataset = dataset.map(self.format_alpaca_dataset)
      tokenized_dataset = dataset.map(lambda example: self.alpaca_tokenize_function(example, tokenizer), batched=True)
      return tokenized_dataset

  def format_alpaca_dataset(self, example):
      try:
          
        SYSTEM_PROMPT = "You are a helpful assistant."
        user_input = example["input"].strip()

        if user_input:
            prompt = (
                "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
                f"{SYSTEM_PROMPT}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                f"{example['instruction']}\n{user_input}<|eot_id|>"
                "<|start_header_id|>assistant<|end_header_id|>\n\n"
            )
        else:
            prompt = (
                "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
                f"{SYSTEM_PROMPT}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                f"{example['instruction']}<|eot_id|>"
                "<|start_header_id|>assistant<|end_header_id|>\n\n"
            )
        completion = example["output"].strip() + "<|eot_id|>"
        result = {"prompt": prompt, "completion": completion}
        return result
      except Exception as e:
          return {}

  def alpaca_tokenize_function(self, example: dict, tokenizer: AutoTokenizer) -> dict:
      tokens = tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=self.dataset_max_length)
      tokens["labels"] = tokens["input_ids"].copy()
      tokens["labels"] = [[(label if label != tokenizer.pad_token_id else -100) for label in labels] for labels in tokens["labels"]]
      tokens["prompt"] = tokens["input_ids"]
      tokens["completion"] = tokens["labels"]
      return tokens


In [8]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(
    model_name
)

model.config.use_cache = False
tokenizer.pad_token = tokenizer.eos_token

Thu Jun 19 17:20:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:01:00.0 Off |                  Off |
| 30%   32C    P8             10W /  300W |      17MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
# peft_config = LoraConfig(
#     task_type="CAUSAL_LM",
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","down_proj","up_proj"]
# )
# model = get_peft_model(model, peft_config)

In [11]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    bf16=True,
    optim="adamw_torch",
    learning_rate=1e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_steps=500,
    num_train_epochs=1,
    logging_steps=1000,
    deepspeed="deepspeed_config_zero2.json",
)

In [12]:
train_util = TrainUtil()
train_dataset, validation_dataset, test_dataset = train_util.load_alpaca_dataset(tokenizer=tokenizer)
tokenized_train_dataset = train_util.load_alpaca_train_dataset(tokenizer=tokenizer, train_dataset=train_dataset)
tokenized_validation_dataset = train_util.load_alpaca_validation_dataset(tokenizer=tokenizer, validation_dataset=validation_dataset)
tokenized_test_dataset = train_util.load_alpaca_test_dataset(tokenizer=tokenizer, test_dataset=test_dataset)

dataset[0]: {'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
dataset[1]: {'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.'}
dataset[2]: {'instruction': 'Describe the structure of an atom.', 'input': '', 'output': 'An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.'}
dataset[3]: {'instruction': 'How can we reduce air pollution?', 'input': '', 'output': 'There are a number of ways to r

Map:   0%|          | 0/8320 [00:00<?, ? examples/s]

Map:   0%|          | 0/8320 [00:00<?, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

In [ ]:
!nvidia-smi

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipykernel_2919726/1536605939.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Authorization required, but no authorization protocol specified

Authorization required, but no authorization protocol specified

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: arekunoimar (arekunoimar-deepspeed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1000,2.588900
2000,2.408200
3000,2.199700
4000,2.148400
5000,2.033200
6000,1.891600
7000,1.807400
8000,1.666700


TrainOutput(global_step=8320, training_loss=2.0741728489215556, metrics={'train_runtime': 4680.0705, 'train_samples_per_second': 1.778, 'train_steps_per_second': 1.778, 'total_flos': 9.949071785171354e+16, 'train_loss': 2.0741728489215556, 'epoch': 1.0})